## Import Libraries

In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
iam = boto3.client('iam')

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()
dependancy_path = 'dependencies.py'
train_data = 's3://aws-sagemaker-one/datasets/input/housing_train_data.csv'

##  Create SageMaker Scikit Estimator 

In [2]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = 'sklearn_entry_point.py'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    framework_version=FRAMEWORK_VERSION,
    instance_type='ml.m4.xlarge',
    dependencies=[dependancy_path],
    sagemaker_session=sagemaker_session)

In [3]:
sklearn_preprocessor.fit({'train': train_data})

2021-04-26 08:16:25 Starting - Starting the training job...
2021-04-26 08:16:49 Starting - Launching requested ML instancesProfilerReport-1619424985: InProgress
......
2021-04-26 08:17:49 Starting - Preparing the instances for training.........
2021-04-26 08:19:22 Downloading - Downloading input data
2021-04-26 08:19:22 Training - Downloading the training image....2021-04-26 08:19:59,945 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-04-26 08:19:59,949 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-26 08:19:59,961 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-04-26 08:20:00,295 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-26 08:20:03,329 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-26 08:20:03,351 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus insta

## Batch transform the training data

In [4]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    assemble_with = 'Line',
    accept = 'text/csv')

In [5]:
# Preprocess training input
transformer.transform(train_data, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path

................................2021-04-26 08:26:09,788 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-04-26 08:26:09,791 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-04-26 08:26:09,792 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;

2021-04-26 08:26:09,788 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-04-26 08:26:09,791 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-04-26 08:26:09,792 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;

  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }

  server {
    list

In [6]:
preprocessed_train

's3://sagemaker-us-east-2-768014633529/sagemaker-scikit-learn-2021-04-26-08-21-05-967'

## Real - Time Transformation of the training data

In [7]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime


timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inference_model = sklearn_preprocessor.create_model()

model_name = 'inference-pipeline-' + timestamp_prefix
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix

#sm_model.sagemaker_session=None
scikit_learn_inference_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name=endpoint_name)

-----------------!

## Make a request to the endpoint

In [10]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, Predictor
#from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

csv_serializer = sagemaker.serializers.CSVSerializer(content_type="text/csv")
data = """-122.26,37.46,26,5067,750,1996,728,7.0001,NEAR OCEAN"""
#-118.15,34.2,52,1786,306,1018,322,4.1518,INLAND
#-118.39,34.12,29,6447,1012,2184,960,8.2816,<1H OCEAN
endpoint_name = endpoint_name

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=csv_serializer)


print(predictor.predict(data))

b'{"instances": [{"features": [-1.3408764930120565, 0.8514634482951091, -0.21099315585118414, 1.1430410231864847, 0.5257730939190086, 0.5164773116405509, 0.6146760713962672, 1.6402561279658028, 0.5819441198401629, -0.030616763948385764, -1.0052566158906469, 0.0, 0.0, 0.0, 0.0, 1.0]}]}'
